In [1]:
%%writefile src/CustomerOrder.sol
//SPDX-License-Identifier:GPL-3.0-or-later
pragma solidity ^0.8.0;

contract CustomerContract {
    struct Customer {
        uint256 id;
        string name;
        string phone;
        string home;
    }

    mapping(address => Customer) customers;
    mapping(uint256 => address) customerIdToAddress;
    mapping(address => uint256) mileage;
    mapping(address => uint256) balanceOf;
    
    function addMileage(address addr, uint amount) public {
        mileage[addr] += amount;
    }

    function refundMileage(address addr, uint amount) public {
        mileage[addr] -= amount;
    }

    function withdrawMoney(address addr, uint num) public payable {
        balanceOf[addr] -= num;
    }

    function refundMoney(address addr, uint num) public payable {
        balanceOf[addr] += num;
    }

    function addCustomer(uint256 _id, string memory _name, string memory _phone, string memory _home) public {
        require(customers[tx.origin].id == 0, "Customer already exists");
        require(customerIdToAddress[_id] == address(0), "Customer ID already exists");

        customers[tx.origin] = Customer(_id, _name, _phone, _home);
        customerIdToAddress[_id] = tx.origin;
        balanceOf[tx.origin] = (tx.origin).balance;
    }

    function getHomeAddress(address _addr) public view returns(string memory) {
        Customer memory m = customers[_addr];
        return m.home;
    }

    function getId() public view returns(uint256) {
        return customers[tx.origin].id;
    }
}

contract OrderContract {
    struct Order {
        uint256 id;
        string product;
        uint256 quantity;
        uint256 amount;
        uint256 timestamp;
        string status;
        string home;
    }

    address owner;
    CustomerContract private c;
    mapping(uint256 => Order) private orders;
    mapping(uint256 => address) private orderIdToCustomer;
    uint256 private orderCount;
    uint private totalOrderAmount;
    uint256[] private orderIds;

    constructor() {
        c = new CustomerContract();
        owner = msg.sender;
    }

    function addMileage(address addr, uint amount) public {
        c.addMileage(addr, amount);
    }

    function refundMileage(address addr, uint amount) public {
        c.refundMileage(addr, amount);
    }

    function withdrawMoney(address addr, uint num)public payable{
        c.withdrawMoney(addr, num);
    }
    
    function refundMoney(address addr, uint num)public payable{
        c.refundMoney(addr, num);
    }

    function addCustomer(uint256 _id, string memory _name, string memory _phone, string memory _home) public {
        c.addCustomer(_id, _name, _phone, _home);
    }

    function getHomeAddress(address _addr) public view returns(string memory) {
        return c.getHomeAddress(_addr);
    }
    
    function placeOrder(uint256 _id, string memory _product, uint256 _quantity, uint256 _amount) public payable{
        require(c.getId() != 0, "Customer does not exist");
        require(orders[_id].id == 0, "Order ID already exists");

        string memory homeaddress = c.getHomeAddress(tx.origin);
        orders[_id] = Order(_id, _product, _quantity, _amount, block.timestamp, "Ordered", homeaddress);
        orderIdToCustomer[_id] = tx.origin;

        addMileage(tx.origin, _amount/100);
        orderCount++;
        totalOrderAmount += _amount;
        require(msg.value == _amount);
        withdrawMoney(tx.origin, _amount);
        orderIds.push(_id);
    }

    function getOrderById(uint256 _id) public view returns (uint256, string memory, string memory, string memory) {
        Order memory o = orders[_id];
        return (o.id, o.product, o.status, o.home);
    }

    function getOrderItem() public view returns (Order[] memory) {
        Order[] memory allOrders = new Order[](orderCount);

        for (uint256 i = 0; i < orderCount; i++) {
            allOrders[i] = orders[orderIds[i]];
        }
        return allOrders;
    }

    function updateStatus(uint _id, string memory _status) public {
        require(orderIdToCustomer[_id] == tx.origin);
        orders[_id].status = _status;
    }

    function getNOrder() public view returns (uint256) {
        return orderCount;
    }

    function getTotalOrderAmount() public view returns (uint256) {
        return totalOrderAmount;
    }

    function queryBalance() public view returns (uint256) {
        return address(this).balance;
    }

    function refund(uint256 _id) public payable {
        require(orderIdToCustomer[_id] == msg.sender);
        orders[_id].status = "Refunded";
        orderCount -= 1;
        totalOrderAmount -= orders[_id].amount;
        refundMileage(tx.origin, orders[_id].amount/100);
        refundMoney(tx.origin, orders[_id].amount);
        payable(owner).transfer(orders[_id].amount);
    }
}

Overwriting src/CustomerOrder.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/CustomerOrder.sol > src/CustomerOrder.json

In [3]:
%%writefile src/CustomerOrderDeploy.js
var Web3 = require('web3');
web3 = new Web3("http://localhost:8345");

_abiBinJson = require('./CustomerOrder.json');
contractName = Object.keys(_abiBinJson.contracts);
//console.log(contractName[1]);
_abi = _abiBinJson.contracts[contractName[1]].abi;
_bin = _abiBinJson.contracts[contractName[1]].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    //########################## 1-2 answer ########################
    await new web3.eth.Contract(_abi).deploy({data: "0x" + _bin}).estimateGas(function(err, gas) {
        if(!err) console.log("gas: " + gas)
    });

    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas:1800000 }, function(err, hash) {
            if(!err) console.log("---> hash: " + hash);
        })
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy();

Overwriting src/CustomerOrderDeploy.js


In [4]:
!node src/CustomerOrderDeploy.js

gas: 1708389
---> hash: 0x37ccbc041877e1c43fedf14bf279c64e6811080e753f813488661bd4184eceed
---> The contract deployed to: 0x84C865B78aD55D89484c2502f9feD64c79898104


In [5]:
%%writefile src/CustomerOrderUse.js
var Web3 = require('web3');
var web3 = new Web3("http://localhost:8345");

var _abiBinJson = require('./CustomerOrder.json');
var contractName = Object.keys(_abiBinJson.contracts);
//console.log(contractName[1]);
var abi = _abiBinJson.contracts[contractName[1]].abi;
var addr = "0x84C865B78aD55D89484c2502f9feD64c79898104";

var order = new web3.eth.Contract(abi,addr);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    //########################## 1-1 answer ########################
    console.log("account[1]: " + accounts[1] + " balance of account[1]: " + await web3.eth.getBalance(accounts[1]));
    console.log("account[2]: " + accounts[2] + " balance of account[2]: " + await web3.eth.getBalance(accounts[2]));
    console.log("account[3]: " + accounts[3] + " balance of account[3]: " + await web3.eth.getBalance(accounts[3]));
    console.log("block number: " + await web3.eth.getBlockNumber());
    

    //########################## 1-3 answer ########################
    await order.methods.addCustomer(111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul").send({from: accounts[1], gas: 1383727});
    await order.methods.addCustomer(112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul").send({from: accounts[2], gas: 1383727});
    await order.methods.addCustomer(113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul").send({from: accounts[3], gas: 1383727});

    //########################## 1-4 answer ########################
    await order.methods.getHomeAddress(accounts[1]).call().then(console.log);
    await order.methods.getHomeAddress(accounts[2]).call().then(console.log);
    await order.methods.getHomeAddress(accounts[3]).call().then(console.log);

    //########################## 1-5 answer ########################
    await order.methods.placeOrder(555, "T-Shirt", 2, 1115).send({from: accounts[1], gas: 1383727, value:1115});
    await order.methods.placeOrder(556, "T-Shirt", 3, 1116).send({from: accounts[2], gas: 1383727, value:1116});
    await order.methods.placeOrder(557, "T-Shirt", 4, 1117).send({from: accounts[3], gas: 1383727, value:1117});

    //########################## 1-6 answer ########################
    await order.methods.getNOrder().call().then(console.log);
    await order.methods.getTotalOrderAmount().call().then(console.log);
    await order.methods.queryBalance().call().then(console.log);

    //########################## 1-7 answer ########################
    await order.methods.updateStatus(556, "On delivery").send({from: accounts[2], gas: 1383727});
    await order.methods.getOrderItem().call().then(function (o) {
        for(i=0; i<3; i++) {
            console.log(o[i].id, o[i].product, o[i].status, o[i].home);
        }
    })

    //########################## 1-8 answer ########################
    await order.methods.getOrderById(556).call().then(function(value) {
        console.log(value[0], value[1], value[2], value[3]);
    });

    //########################## 1-9 answer ########################
    await order.methods.refund(556).send({from: accounts[2]});

    //########################## 1-10 answer ########################
    await order.methods.getOrderById(556).call().then(function(value) {
        console.log(value[0], value[1], value[2], value[3]);
    });

    await order.methods.getNOrder().call().then(console.log);
    await order.methods.getTotalOrderAmount().call().then(console.log);
    await order.methods.queryBalance().call().then(console.log);
}
doIt()

Overwriting src/CustomerOrderUse.js


In [6]:
!node src/CustomerOrderUse.js

account[1]: 0xa9643989300947AC769B56E51E27fa8ec19fa399 balance of account[1]: 999992543017771412429
account[2]: 0x8aDEAe08913b985ddec97e6a50c7e89C2492BBbC balance of account[2]: 999993447440460340256
account[3]: 0x21189730f8dadC1D6453463589159D30f9715A47 balance of account[3]: 999994261217494050839
block number: 57
111 hongji-dong jongro-gu seoul
112 hongji-dong jongro-gu seoul
113 hongji-dong jongro-gu seoul
3
3348
3348
555 T-Shirt Ordered 111 hongji-dong jongro-gu seoul
556 T-Shirt On delivery 112 hongji-dong jongro-gu seoul
557 T-Shirt Ordered 113 hongji-dong jongro-gu seoul
556 T-Shirt On delivery 112 hongji-dong jongro-gu seoul
556 T-Shirt Refunded 112 hongji-dong jongro-gu seoul
2
2232
2232
